In [63]:
import random
import numpy as np
import json
import pandas as pd

import os
import datetime


In [12]:
from faker import Faker
fake = Faker()

In [35]:
Faker.seed(0)

In [79]:
sensTypes = {
    "pressure": {
        "sampleRate": 3600,
        "dim": [
            {"name": "P", "range": [100, 200]}
        ]
    }, 
    "temperature":{
        "sampleRate": 3600,
        "dim": [
            {"name":"T", "range": [800, 1200]}
        ]
    }, 
    "vibration": {
        "sampleRate": 60,
        "dim": [
            {"name":"X", "range": [-1, 1]}, 
            {"name":"Vel", "range": [-1, 1]}, 
            {"name":"Acc", "range": [-1,1]}
        ]
    }
}

In [53]:
conditions = {
    "normal":None, 
    "unnormal":None, 
    "faulty": None, 
    "critical":None, 
    "shutdown":None
}

In [72]:
def addSecs(tm, secs = 60):
    fulldate = datetime.datetime(tm., 1, 1, tm.hour, tm.minute, tm.second)
    fulldate = fulldate + datetime.timedelta(seconds=secs)
    return fulldate

In [58]:
def selectRandItem(items):
    
    lst_keys = list(items.keys())
    return lst_keys[random.randint(0, len(lst_keys) - 1)]
    

In [92]:
def initiate_sensor(machineId, num=10):
    
    lst_sensors = []
    for i in range(num):
        lst_sensors.append({
            "sensId": f"{machineId}_{i+1}",
            "senstype": selectRandItem(sensTypes),
            "name": fake.domain_word(),
            "tolerance": random.uniform(0.001, 0.000001),
            "histData": []
        })

    return lst_sensors

In [89]:
def initiate_machine(num = 10):
    
    lst_machines = []    
    for i in range(num):
        latlng = fake.latlng()
        num_sens = random.randint(10, 20)
        mach = {
            "coordinate": {"lat": float(latlng[0]), "lng": float(latlng[1])},
            "machId": f"{i + 1}",
            "name": fake.domain_word(),
            "state": {
                "condition": selectRandItem(conditions),
                "availability": bool(random.randint(0,1)),
                "powerproduction": []
            }
        }
        
        mach["sensors"] = initiate_sensor(mach["machId"], num= num_sens)
        
        lst_machines.append(mach)
        
    return lst_machines

# Setup Machines and Sensors

In [104]:
lst_machs = initiate_machine(num = 15)

# Generate Sensor Data for Machines

In [99]:
def generate_sens_data(currSensType, currTime):
    
    sampleRate = sensTypes[currSensType]["sampleRate"]
    
    val = {}
    if currTime%sampleRate == 0:        
        for sens in sensTypes[currSensType]["dim"]:
            minVal = sens["range"][0]
            maxVal = sens["range"][1]
            val[sens["name"]] = random.uniform(minVal, maxVal)
            
    return val
    

In [105]:
num_days         = 1
sampleDate       = datetime.datetime(2020,1,1,12,0,0)
maxSampleTimeSec = num_days * 24 * 3600

powerprod_interval = 3600
powerprod_range    = [50, 100]

tot_time   = 0
delta_time = 60
while (tot_time < maxSampleTimeSec):
    
    for mach in lst_machs:
        
        if tot_time%powerprod_interval:
            
            mach["state"]["powerproduction"].append({
                "timestamp": str(sampleDate),
                "power": random.uniform(
                    powerprod_range[0], 
                    powerprod_range[1]
                )
            })
            
            for sens in mach["sensors"]:
                sens_val = generate_sens_data(sens["senstype"], tot_time)
                if (sens_val):
                    sens["histData"].append({
                        "timestamp": str(sampleDate),
                        "value": sens_val
                    })
    
    tot_time += delta_time
    sampleDate +=  datetime.timedelta(seconds=delta_time)

In [107]:
with open('sensordata.json', 'w') as fp:
    json.dump(lst_machs, fp, sort_keys=True, indent=4, separators=(',', ': '))